# Setting up inputs for the gridded DBPM run
**Author**: Denisse Fierro Arcos  
**Date**: 2025-01-10  
  
In this notebook, we will prepare all necessary inputs need to run the gridded version of DBPM. Note that this step needs to be completed only once for each model run.  

## Loading relevant libraries

In [1]:
import os
os.chdir('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/new_workflow/')
from glob import glob
import xarray as xr
import json
import pandas as pd
import numpy as np
import useful_functions as uf
from dask.distributed import Client

## Start a cluster for parallelisation

In [2]:
client = Client(threads_per_worker = 1)

## Defining output folder

In [2]:
base_folder = '/g/data/vf71/la6889/dbpm_inputs/east_antarctica/'
model_res = '025deg'
reg_int = 'fao-58'
out_folder = os.path.join(base_folder, 'gridded_params', model_res)
os.makedirs(out_folder, exist_ok = True)

## Transforming DBPM parameters to Python-friendly format
These parameters are the outputs of the `sizeparam` function for R. They were calculated and stored in the script ([04_calculating_dbpm_fishing_params](new_workflow/04_calculating_dbpm_fishing_params.R)) as a `json` file.

In [3]:
#Loading gridded parameters
gridded_params = json.load(open(
    os.path.join(out_folder, f'dbpm_gridded_size_params_{reg_int}.json')))

#Transforming them to Python friendly format
gridded_python = uf.gridded_param_python(gridded_params)

### Adding useful entries to gridded parameters
These variables are used multiple times within the `gridded_size_model` script, so they will be added for convenience.

In [4]:
# Size classes
log10_size_bins = np.array(gridded_python['log10_size_bins'])
# Minimum (log10) size class for predators
log10_ind_min_pred_size = log10_size_bins[gridded_python['ind_min_pred_size']]
ind_min_detritivore_size = gridded_python['ind_min_detritivore_size']
log10_ind_min_detritivore_size = log10_size_bins[ind_min_detritivore_size]
log10_ind_min_fish_pred = log10_size_bins[gridded_python['ind_min_fish_pred']]
log10_ind_min_fish_det = log10_size_bins[gridded_python['ind_min_fish_det']]

#Adding new variables to gridded parameters
gridded_python['log10_ind_min_pred_size'] = log10_ind_min_pred_size
gridded_python['log10_ind_min_detritivore_size'] = log10_ind_min_detritivore_size

### Saving gridded parameters

In [5]:
out_gridded = os.path.join(out_folder, f'dbpm_gridded_size_params_{reg_int}_python.json')

#Save to disk
with open(out_gridded, 'w') as outfile:
    json.dump(gridded_python, outfile)

## Loading gridded input data
Gridded data are needed both as direct input to DBPM and to calculate params, such as habitat preference, pelagic predator and benthic detritivore size spectrum, among other things.

In [5]:
depth = xr.open_zarr(glob(
    os.path.join(base_folder, 'gridded', model_res, 
                 f'gfdl-mom6-cobalt2_obsclim_deptho_*_{reg_int}_fixed.zarr/'))[0])['deptho']

In [7]:
#Loading intercept data for spinup period
int_phy_zoo_spinup = xr.open_zarr(glob(
    os.path.join(base_folder, 'gridded', model_res,
                 f'gfdl-mom6-cobalt2_spinup_intercept_*_{reg_int}_monthly_1841_1960.zarr/'))[0])['intercept']

#Loading intercept data for model period and fixing time values
int_phy_zoo  = xr.open_zarr(glob(
    os.path.join(base_folder, 'gridded', model_res,
                 f'gfdl-mom6-cobalt2_obsclim_intercept_*_{reg_int}_monthly_1961_2010.zarr/'))[0])['intercept']
int_phy_zoo['time'] = pd.DatetimeIndex(int_phy_zoo.indexes['time'].to_datetimeindex(),
                                         dtype = 'datetime64[ns]')

In [8]:
#Loading slope data for spinup period
slope_phy_zoo_spinup = xr.open_zarr(glob(
    os.path.join(base_folder, 'gridded', model_res,
                 f'gfdl-mom6-cobalt2_spinup_slope_*_{reg_int}_monthly_1841_1960.zarr/'))[0])['slope']

#Loading slope data for model period and fixing time values
slope_phy_zoo  = xr.open_zarr(glob(
    os.path.join(base_folder, 'gridded', model_res,
                 f'gfdl-mom6-cobalt2_obsclim_slope_*_{reg_int}_monthly_1961_2010.zarr/'))[0])['slope']
slope_phy_zoo['time'] = pd.DatetimeIndex(slope_phy_zoo.indexes['time'].to_datetimeindex(),
                                         dtype = 'datetime64[ns]')

In [9]:
#Loading sea surface temperature data for spinup period
sea_surf_temp_spinup = xr.open_zarr(glob(
    os.path.join(base_folder, 'gridded', model_res,
                 f'gfdl-mom6-cobalt2_spinup_tos_*_{reg_int}_monthly_1841_1960.zarr'))[0])['tos']

#Loading sea surface temperature data for model period and fixing time values
sea_surf_temp  = xr.open_zarr(glob(
    os.path.join(base_folder, 'gridded', model_res,
                 f'gfdl-mom6-cobalt2_obsclim_tos_*_{reg_int}_monthly_1961_2010.zarr/'))[0])['tos']
sea_surf_temp['time'] = pd.DatetimeIndex(sea_surf_temp.indexes['time'].to_datetimeindex(),
                                         dtype = 'datetime64[ns]')

/jobfs/132832343.gadi-pbs/ipykernel_3574228/3153113770.py:10: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  sea_surf_temp['time'] = pd.DatetimeIndex(sea_surf_temp.indexes['time'].to_datetimeindex(),


In [10]:
#Loading bottom ocean temperature data for spinup period
sea_floor_temp_spinup = xr.open_zarr(glob(
    os.path.join(base_folder, 'gridded', model_res,
                 f'gfdl-mom6-cobalt2_spinup_tob_*_{reg_int}_monthly_1841_1960.zarr'))[0])['tob']

#Loading bottom ocean temperature data for model period and fixing time values
sea_floor_temp  = xr.open_zarr(glob(
    os.path.join(base_folder, 'gridded', model_res,
                 f'gfdl-mom6-cobalt2_obsclim_tob_*_{reg_int}_monthly_1961_2010.zarr/'))[0])['tob']
sea_floor_temp['time'] = pd.DatetimeIndex(sea_floor_temp.indexes['time'].to_datetimeindex(),
                                          dtype = 'datetime64[ns]')

/jobfs/132832343.gadi-pbs/ipykernel_3574228/3281048223.py:10: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  sea_floor_temp['time'] = pd.DatetimeIndex(sea_floor_temp.indexes['time'].to_datetimeindex(),


### Creating time variable that contains a timestep for initialisation of DBPM
This will only be applied to the `spinup` period. It will add a timestep before the start of the period to initialise DBPM. This is not applied to the modelling period because values from the last time step of the `spinup` period will be used to run DBPM.

In [9]:
time_spinup = (np.append((int_phy_zoo_spinup.time.min().values-np.timedelta64(1, 'm')),
                         int_phy_zoo_spinup.time.values))

## Creating gridded variables needed for DBPM run
We will create data array versions of the gridded inputs.

### DBPM size classes

In [10]:
# Log10 size of individuals found in the model
log10_size_bins_mat = xr.DataArray(data = log10_size_bins,
                                   dims = ['size_class'], 
                                   coords = {'size_class': log10_size_bins})

# size of individuals found in the model
size_bin_vals = 10**log10_size_bins_mat

### Fishing effort
Effort data for the `spinup` and modelling period is provided in a single entry in `gridded_python`. We will split this data into the `spinup` and modelling period.

In [13]:
effort_spinup = (xr.DataArray(gridded_python['effort'][:len(int_phy_zoo_spinup.time)], 
                              dims = 'time', 
                              coords = {'time': int_phy_zoo_spinup.time}).
                 expand_dims({'lat': depth.lat, 'lon': depth.lon}).
                 transpose('time', 'lat', 'lon')).where(np.isfinite(depth))

effort = (xr.DataArray(gridded_python['effort'][len(int_phy_zoo_spinup.time):],
                       dims = 'time', coords = {'time': int_phy_zoo.time}).
          expand_dims({'lat': depth.lat, 'lon': depth.lon}).
          transpose('time', 'lat', 'lon')).where(np.isfinite(depth))

### Setting habitat preferences
Predator coupling to benthic prey, which is depth dependent, 0.75 above 500 m, 0.5 between 500 and 1800 m, and 0 below 1800 m. Predator coupling to pelagic prey is equal to `1-(benthic coupling)`.  
  
These values were suggested by Clive Trueman based on stable isotope work, and proportion of biomass. Rockall Trough studies.

In [14]:
pref_benthos = (0.8*np.exp((-1/250*depth)))
pref_pelagic = (1-pref_benthos)

### Initiliasing values for pelagic predators and benthic detritivores

In [11]:
init_pred = (xr.DataArray(data = gridded_python['init_pred'], 
                          dims = ['size_class'], 
                          coords = {'size_class': log10_size_bins}).
             expand_dims({'lat': depth.lat, 'lon': depth.lon}))

init_detritivores = (xr.DataArray(data = gridded_python['init_detritivores'], 
                                  dims = ['size_class'], 
                                  coords = {'size_class': log10_size_bins}).
                     expand_dims({'lat': depth.lat, 'lon': depth.lon}))

### Calculating constant growth and mortality

In [16]:
pred_prey_mat = uf.pred_prey_matrix(log10_size_bins)

constant_growth = xr.DataArray(uf.gphi_f(pred_prey_mat, 
                                         gridded_python['log10_pred_prey_ratio'],
                                         gridded_python['log_prey_pref']),
                               dims = ['size_class', 'sc'])

constant_mortality = xr.DataArray(uf.mphi_f(-pred_prey_mat, 
                                            gridded_python['log10_pred_prey_ratio'],
                                            gridded_python['log_prey_pref'],
                                            gridded_python['metabolic_req_pred']),
                                  dims = ['size_class', 'sc'])

### Building metabolic requirements lookup table 
Metabolic requirements are estimated per size class

In [17]:
met_req_log10_size_bins = uf.expax_f(log10_size_bins_mat, 
                                     gridded_python['metabolic_req_pred'])

In [18]:
consume_pelagic = (pref_pelagic*gridded_python['hr_volume_search']*
                   met_req_log10_size_bins)
consume_benthos = (pref_benthos*gridded_python['hr_volume_search']*
                   met_req_log10_size_bins)

### Creating a gridded time series of intercept of plankton size spectrum 
The size spectrum was estimated by the `GetPPIntSlope` function using GFDL-MOM6-COBALT2 outputs in script ([01_processing_dbpm_global_inputs](new_workflow/01_processing_dbpm_global_inputs.ipynb)).

In [12]:
ui0_spinup = 10**int_phy_zoo_spinup
ui0 = 10**int_phy_zoo

### Calculating `predator` biomass for `spinup` and modelling period
Values are derived from plankton-zooplankton size spectrum, slope and size class.

In [13]:
# Calculating predator biomass for spinup period
pred_start_spinup = ui0_spinup*(10**(slope_phy_zoo_spinup*log10_size_bins_mat))
# Keep values outside the predator size classes, otherwise change to zero (0)
pred_start_spinup = xr.where(pred_start_spinup.size_class <= log10_ind_min_pred_size,
                             pred_start_spinup, 0)
# Copy first time step predator values as 'initialisation' timestep
pred_start_spinup = xr.concat([pred_start_spinup.isel(time = 0), pred_start_spinup],
                              dim = 'time')
# Update time labels
pred_start_spinup['time'] = time_spinup
# Assigning initial values (time = 0) for predators from gridded parameters
predators_spinup = xr.where((pred_start_spinup.time == pred_start_spinup.time.min()) & 
                            (pred_start_spinup.size_class >= log10_ind_min_pred_size),
                            init_pred, pred_start_spinup)
#Applying spatial mask
predators_spinup = predators_spinup.where(np.isfinite(depth))

# Calculating predator biomass for modelling period
pred_start = ui0*(10**(slope_phy_zoo*log10_size_bins_mat))
# Keep values outside the predator size classes, otherwise change to zero (0)
predators = xr.where(pred_start.size_class <= log10_ind_min_pred_size,
                      pred_start, 0)
#Applying spatial mask
predators = predators.where(np.isfinite(depth))

### Detritivore biomass and detritus
Creating gridded versions of `detritivores` and `detritus` initalising values to be used in gridded DBPM.

In [21]:
# Assigning initial values (time = 0) for detritivores from gridded parameters
detritivores = xr.where((init_detritivores.size_class >= log10_ind_min_detritivore_size),
                        init_detritivores, 0)
# Adding time dimension
detritivores = detritivores.expand_dims({'time': [min(time_spinup)]})

#Applying spatial mask
detritivores = detritivores.where(np.isfinite(depth))

# Initialising detritus
detritus = xr.zeros_like(detritivores.isel(size_class = 0).drop_vars('size_class'))
# Assigning initial values (time = 0) for detritus from gridded parameters
detritus = xr.where(detritus == 0, gridded_python['init_detritus'], detritus)
#Applying spatial mask
detritus = detritus.where(np.isfinite(depth))

### Note
If no initial values for `predators` and `detritivores` are included in the gridded parameters, use the following code instead:  
  
```
predators_spinup = xr.where((pred_start_spinup.time == pred_start_spinup.time.min()) &
                       (pred_start_spinup.size_class >= log10_ind_min_pred_size) &
                       (pred_start_spinup.size_class < log10_size_bins[120]),
                       init_pred, pred_start_spinup)

detritivores = xr.where((init_detritivores.size_class >= log10_ind_min_detritivore_size) &
                         (init_detritivores.size_class < log10_size_bins[120]), 
                         init_detritivores, detri_init)
```

### Fishing mortality
Here `fishing_mort_pred` and `fishing_mort_det` are fixed catchability terms for `predators` and `detritivores`. Fishing mortality is estimated only for individuals within fished sizes (`ind_min_fish_pred` or `ind_min_fish_det`).

In [22]:
# Using fish_mort_pred as a base for fishing mortality for predators
fish_mort_pred = xr.DataArray(
    np.repeat(gridded_python['fish_mort_pred'], 
              np.prod(consume_pelagic.shape)).reshape(consume_pelagic.shape),
    dims = ['lat', 'lon', 'size_class'],
    coords = {'lat': consume_pelagic.lat, 'lon': consume_pelagic.lon,
             'size_class': consume_pelagic.size_class})

# Changing mortality to zero outside the predator size fished
fishing_mort_pred = xr.where((fish_mort_pred.size_class >= log10_ind_min_fish_pred) &
                          (fish_mort_pred.size_class < 
                           fish_mort_pred.size_class.max()),
                          fish_mort_pred, 0).where(np.isfinite(depth))

In [23]:
# Using fish_mort_det as a base for fishing mortality for detritivores
fish_mort_det = xr.DataArray(
    np.repeat(gridded_python['fish_mort_detritivore'], 
              np.prod(consume_benthos.shape)).reshape(consume_benthos.shape),
    dims = ['lat', 'lon', 'size_class'],
    coords = {'lat': consume_benthos.lat, 'lon': consume_benthos.lon,
             'size_class': consume_benthos.size_class})

# Changing mortality to zero outside the detritivore size fished
fishing_mort_det = xr.where((fish_mort_det.size_class >= log10_ind_min_fish_det) &
                          (fish_mort_det.size_class < 
                           fish_mort_det.size_class.max()),
                          fish_mort_det, 0).where(np.isfinite(depth))

### Temperature effect on pelagic organisms
This will be calculated for the `spinup` and modelling periods

In [24]:
pel_tempeffect_spinup = np.exp(gridded_python['c1']-gridded_python['activation_energy']/
                               (gridded_python['boltzmann']*(sea_surf_temp_spinup+273)))

pel_tempeffect = np.exp(gridded_python['c1']-gridded_python['activation_energy']/
                        (gridded_python['boltzmann']*(sea_surf_temp+273)))

### Temperature effect on benthic organisms
This will be calculated for the `spinup` and modelling periods

In [25]:
ben_tempeffect_spinup = np.exp(gridded_python['c1']-gridded_python['activation_energy']/
                               (gridded_python['boltzmann']*(sea_floor_temp_spinup+273)))

ben_tempeffect = np.exp(gridded_python['c1']-gridded_python['activation_energy']/
                        (gridded_python['boltzmann']*(sea_floor_temp+273)))

### Note
If `temp_effect` is not needed, then simply assign `1` to `pel_tempeffect` and `ben_tempeffect`:  
  
```
pel_tempeffect = 1
ben_tempeffect = 1
```

## Saving gridded inputs
We will now save all the gridded inputs needed to run the DBPM model.  

In [14]:
#Get the correct model resolution in arcmin to add to file name
if model_res == '1deg':
    model_res_arc = '60arcmin'
elif model_res == '025deg':
    model_res_arc = '15arcmin'

The `log10_size_bins_mat` only needs to be saved once per because size classes are the same for a set of experiments. This variable does **not** vary across space or time.

In [28]:
log10_size_bins_mat.name = 'size_bins'
log10_size_bins_mat.to_zarr('outputs/log10_size_bins_matrix.zarr/',
                            consolidated = True, mode = 'w')

**Note:** The best chunk size will vary across different regions and variables. As a rule of thumb, chunk size should be kept between 100 MB and 1GB. Chunking will also need to consider the calculations applied to the data. For example, if calculations need to be applied to the entire area of interest at each time step, it may be best to chunk data along the `time` dimension, but not along `latitude` and `longitude`.

In [29]:
effort_spinup.name = 'effort'
effort_out = effort_spinup.chunk({'lon': len(effort_spinup.lon), 
                                  'lat': len(effort_spinup.lat), 'time': 120})
effort_out.to_zarr(
    os.path.join(out_folder, f'effort_spinup_{model_res_arc}_{reg_int}_monthly_1841_1960.zarr/'),
                   consolidated = True, mode = 'w')

effort.name = 'effort'
effort_out = effort.chunk({'lon': len(effort.lon), 'lat': len(effort.lat), 'time': 120})
effort_out.to_zarr(
    os.path.join(out_folder, f'effort_{model_res_arc}_{reg_int}_monthly_1961_2010.zarr/'),
                   consolidated = True, mode = 'w')

In [30]:
pref_benthos.name = 'pref_benthos'
pref_benthos.to_zarr(os.path.join(out_folder, 
                                  f'pref-benthos_{model_res_arc}_{reg_int}_fixed.zarr/'),
                     consolidated = True, mode = 'w')
pref_pelagic.name = 'pref_pelagic'
pref_pelagic.to_zarr(os.path.join(out_folder, 
                                  f'pref-pelagic_{model_res_arc}_{reg_int}_fixed.zarr/'), 
                     consolidated = True, mode = 'w')

In [31]:
constant_growth.name = 'constant_growth'
constant_growth.to_zarr(os.path.join(out_folder,
                                     f'const-growth_{model_res_arc}_{reg_int}_fixed.zarr/'),
                        consolidated = True, mode = 'w')

constant_mortality.name = 'constant_mortality'
constant_mortality.to_zarr(os.path.join(out_folder,
                                        f'const-mort_{model_res_arc}_{reg_int}_fixed.zarr/'),
                        consolidated = True, mode = 'w')

In [32]:
consume_pelagic.name = 'consume_pelagic'
consume_pelagic.to_zarr(os.path.join(out_folder,
                                     f'consume-pelagic_{model_res_arc}_{reg_int}_fixed.zarr/'),
                        consolidated = True, mode = 'w')

consume_benthos.name = 'consume_benthos'
consume_benthos.to_zarr(os.path.join(out_folder,
                                     f'consume-benthos_{model_res_arc}_{reg_int}_fixed.zarr/'),
                        consolidated = True, mode = 'w')

In [33]:
detritivores.name = 'detritivores'
detritivores.to_zarr(os.path.join(out_folder, 
                                  f'detritivores_{model_res_arc}_{reg_int}_monthly_1961_2010.zarr/'),
                     consolidated = True, mode = 'w')

detritus.name = 'detritus'
detritus.to_zarr(os.path.join(out_folder, 
                              f'detritus_{model_res_arc}_{reg_int}_monthly_1961_2010.zarr/'),
                 consolidated = True, mode = 'w')

In [34]:
fishing_mort_pred.name = 'fish_mort_pred'
fishing_mort_pred.to_zarr(os.path.join(out_folder, 
                                       f'fish-mort-pred_{model_res_arc}_{reg_int}_monthly_1961_2010.zarr/'),
                          consolidated = True, mode = 'w')

fishing_mort_det.name = 'fish_mort_det'
fishing_mort_det.to_zarr(os.path.join(
    out_folder, f'fish-mort-det_{model_res_arc}_{reg_int}_monthly_1961_2010.zarr/'),
                         consolidated = True, mode = 'w')

In [35]:
pel_tempeffect_spinup.name = 'pel_temp_eff' 
pel_tempeffect_out = pel_tempeffect_spinup.chunk({'lat': 106, 'lon': 480})
pel_tempeffect_out.to_zarr(
    os.path.join(out_folder, 
                 f'pel-temp-eff_spinup_{model_res_arc}_{reg_int}_monthly_1841_1960.zarr/'),
    consolidated = True, mode = 'w')

pel_tempeffect.name = 'pel_temp_eff' 
pel_tempeffect_out = pel_tempeffect.chunk({'lat': 106, 'lon': 480})
pel_tempeffect_out.to_zarr(
    os.path.join(out_folder, f'pel-temp-eff_{model_res_arc}_{reg_int}_monthly_1961_2010.zarr/'),
    consolidated = True, mode = 'w')

In [36]:
ben_tempeffect_spinup.name = 'ben_temp_eff'
ben_tempeffect_out = ben_tempeffect_spinup.chunk({'lat': 106, 'lon': 480})
ben_tempeffect_out.to_zarr(
    os.path.join(out_folder, 
                 f'ben-temp-eff_spinup_{model_res_arc}_{reg_int}_monthly_1841_1960.zarr/'),
    consolidated = True, mode = 'w')

ben_tempeffect.name = 'ben_temp_eff'
ben_tempeffect_out = ben_tempeffect.chunk({'lat': 106, 'lon': 480})
ben_tempeffect_out.to_zarr(
    os.path.join(out_folder, f'ben-temp-eff_{model_res_arc}_{reg_int}_monthly_1961_2010.zarr/'),
    consolidated = True, mode = 'w')

### Predator data
Predator data are four dimensional files and they are too large to save as a single file, so it is best to split them along the time dimension. Dividing them into 5-year chunks makes saving them faster for regions like the Weddell and East Antarctica. For larger regions, it may be necessary to split the data into smaller yearly files.  
  
In this section we show how to save files into yearly and 5-yearly files. **Note** that the chunk sizes may need to be changed based on the dimensions of your data.

#### Five-year files

In [31]:
#Name data array
predators_spinup.name = 'predators'

#Group data every 5 years
yrs = predators_spinup.time.dt.year
half_dec = ((yrs - yrs[0])/5).astype(int)
predators_spinup = predators_spinup.assign_coords({'half_dec': half_dec})

#Save data
for i, da in predators_spinup.groupby('half_dec'):
    ymin = str(da.time.dt.year.min().values)
    ymax = str(da.time.dt.year.max().values)
    da_out = da.chunk({'lon': int(len(da.lon)/3), 'lat': len(da.lat),
                       'time': 24}).drop_vars('half_dec')
    da_out.to_zarr(
        os.path.join(out_folder, 
                     f'predators_spinup_{model_res_arc}_{reg_int}_monthly_{ymin}_{ymax}.zarr/'),
        consolidated = True, mode = 'w')

In [ ]:
#Name data array
predators.name = 'predators'

#Group data every 5 years
yrs = predators.time.dt.year
half_dec = ((yrs - yrs[0])/5).astype(int)
predators = predators.assign_coords({'half_dec': half_dec})

#Save data
for i, da in predators.groupby('half_dec'):
    ymin = str(da.time.dt.year.min().values)
    ymax = str(da.time.dt.year.max().values)
    da_out = da.chunk({'lon': int(len(da.lon)/4), 'lat': len(da.lat),
                       'time': 12}).drop_vars('half_dec')
    da_out.to_zarr(
        os.path.join(out_folder, f'predators_{model_res_arc}_{reg_int}_monthly_{ymin}_{ymax}.zarr/'),
        consolidated = True, mode = 'w')

#### Yearly files

In [ ]:
#Name data array
predators_spinup.name = 'predators'

#Split data into yearly files
for yr, da in predators_spinup.groupby('time.year'):
    da_out = da.chunk({'lon': len(da.lon), 'lat': len(da.lat), 'time': 2})
    da_out.to_zarr(
        os.path.join(out_folder, 
                     f'predators_spinup_{model_res_arc}_{reg_int}_monthly_{yr}.zarr/'),
        consolidated = True, mode = 'w')

In [14]:
#Name data array
predators.name = 'predators'

#Split data into yearly files
for yr, da in predators.groupby('time.year'):
    da_out = da.chunk({'lon': len(da.lon), 'lat': len(da.lat), 'time': 2})
    da_out.to_zarr(
        os.path.join(out_folder, f'predators_{model_res_arc}_{reg_int}_monthly_{yr}.zarr/'),
        consolidated = True, mode = 'w')